# Feedforward Nerual Network ( FNN )

Feedforward is the simplest form of neural network. 
The neural network passes infromation as a one way direction feed from to the input neurons and passed through 
every hidden layer until ending up at the output layer where the vector is returned.
What makes a feedforward neural network is that it does not cycle bewteen connections as recurrent neural network do.
While data will go through multiple layers it will only happend to end up at that layer ones and then be feed forward to the next.

## Activation Function
In an artifical neural network, the activation function of a neuron defines the output of that neuron given a set of weighted sum of its input. 
These weighted sum of its input values which are feed to the activation function. 

A activation function transform the output value to a state of logical operation
for the next neuron to interpert if using the output or discarding the output.

### ReLU
ReLU is a simple activation function that is used to determine if the neuron 
should use the value. 
It achives this by reciving a numerical value $z$ where $z \in R$ and have been
computed from $z = w*x+b$. The z is feed to the ReLU function:
$$ReLU(z)= max(0,z)$$
What this function state is that if a x value is less than zero then return zero.
Otherwise $z = ReLU(z)$.

### Sigmoid
Sigmoid functions is a logistic function.
Logistic regression is a technique to predict the otucome of binary classifictation problems.
In the case of a activation function it then takes a weighted sum of the input features as
an input and outputs the probability value of the outcome.

The mathematical formula of sigmoid:
$$S(z) = \frac{1}{1 + e^{-z}} $$


## Neural Network

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [24]:
class NeuralNetwork(nn.Module):
    def __init__(self, n_inputs=2, h_layer=3, n_neurons=10, n_output=1):
        super().__init__()

        self.activation = nn.Sigmoid()
        
        # Input layer
        self.input_layer = nn.Linear(n_inputs, n_neurons) # Inputs 1 card

        # Hidden layers
        self.hidden_layer = [nn.Linear(n_neurons, n_neurons) for _ in range(h_layer)]

        # Output layer
        self.output_layer = nn.Linear(n_neurons, n_output) # Output: call or fold -> 2 classes call, fold

    def forward(self, x):
        x = F.relu(self.input_layer(x))

        for layer in self.hidden_layer:
            x = F.relu(layer(x))

        output = self.output_layer(x)
        
        return self.activation(output)

## Kuhn Poker
Kuhn poker is an extremely simplified form of poker.
Kuhn as a simple model zero-sum two-player imperfect-information game. 
Which has a nash equilbrium strategy in playing this game.<br>
In Kuhn poker, the deck includes only three playing cards, in this deck there will be a King, Queen and Jack. 

* Play:
    - One card is dealt to each player, which may place bets similarly to a standard poker. Both player have now the option to either bet or pass.
    - If both players bet or both players pass, the player with the higher card wins, otherwise, the betting player wins.

In [7]:
from kuhn import Kuhn

In [8]:
# Setup
kuhn = Kuhn()
kuhn.new_match("human","bot")

# First round / Pre showdown
hands = kuhn.pre_showdown()

# Our hand
print("Our human card", hands["human"])

# Showdown
winner = kuhn.showdown()
print(f"\nHands human: {hands['human']} and bot: {hands['bot']}", "\nWinner is:", winner)

Our human card J

Hands human: J and bot: K 
Winner is: bot


## Training

In [33]:
def card_compute(card, hand):
    return Kuhn.deck_value[hands]

def expected_outcome(hands, player, opponent):
    prophecy = {('J', 'Q'): (0, 0), 
                ('J', 'K'): (0, 0), 
                ('Q', 'J'): (1, 1),
                ('Q', 'K'): (1, 0),
                ('K', 'J'): (1, 1),
                ('K', 'Q'): (1, 1)}
    cards = (hands[player], hands[opponent])
    return prophecy[cards]

def clear_output(output):
    return 1 if output > 0.5 else 0

def reward(expected_outcome, outcome_all):
    REWARD_CONSTANT = 0.3 # Used to control reward
    clear_outcome_all = [clear_output(outcome) for outcome in outcome_all]
    idx = next((i for i in range(len(outcome_all)) if clear_outcome_all[i] != expected_outcome[i]), len(outcome_all))
    return  -REWARD_CONSTANT if (len(outcome_all) - idx) == 0 else (len(outcome_all) - idx)*REWARD_CONSTANT # Reward or Penalty

In [13]:
kuhn = Kuhn()
for _ in range(3):
    print(5*"=","New Match", "="*5)
    kuhn.new_match("human", "bot")
    hands = kuhn.pre_showdown()
    print(hands, expected_outcome(hands, "bot", "human"))
    winner = kuhn.showdown()
    print("showdown:", winner)

===== New Match =====
{'human': 'J', 'bot': 'Q'} (1, 1)
showdown: bot
===== New Match =====
{'human': 'J', 'bot': 'Q'} (1, 1)
showdown: bot
===== New Match =====
{'human': 'J', 'bot': 'Q'} (1, 1)
showdown: bot


In [36]:
def train(model, epochs):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    loss = nn.BCELoss()
    kuhn = Kuhn()

    for epoch in range(epochs):
        print(5*"=", f"Epoch {epoch+1}", "="*5)
        # Game new match setup
        kuhn.new_match("human", "bot")
        hands = kuhn.pre_showdown()
        print(f"HANDS: bot: {hands['bot']}, human: {hands['human']}")
        
        model.zero_grad()

        y = torch.tensor(expected_outcome(hands, "bot", "human"), dtype=torch.float)

        outcome_all = []
        for i in range(2):
            X = torch.tensor([Kuhn.deck_value[hands["bot"]], i], dtype=torch.float, requires_grad=True)
            output = model(X)
            outcome_all.append(output)
    
            # FOLD
            if clear_output(output) == 0: break

        if len(outcome_all) == 2:
            winner = kuhn.showdown()
            print(f"Winner: {winner}")
        else:
            print(f'Folded round: {len(outcome_all)}')
            outcome_all.append(torch.tensor([0.], dtype=torch.float, requires_grad=True))
        
        outcome_all = torch.cat(outcome_all)
        print(f"outcome all: {outcome_all}")
        
        output = loss(outcome_all, y)

        # Introduce penalty and reward
        output += reward(y, outcome_all)

        print(f'Loss: {output}')
        
        output.backward()
        optimizer.step()

### Training

In [37]:
EPOCHS = 10
net = NeuralNetwork()
print(5*"=", f"Neural Network", "="*5)
print(net)
train(net, EPOCHS)

===== Neural Network =====
NeuralNetwork(
  (activation): Sigmoid()
  (input_layer): Linear(in_features=2, out_features=10, bias=True)
  (output_layer): Linear(in_features=10, out_features=1, bias=True)
)
===== Epoch 1 =====
HANDS: bot: K, human: J
Folded round: 1
outcome all: tensor([0.3773, 0.0000], grad_fn=<CatBackward0>)
Loss: 51.08730697631836
===== Epoch 2 =====
HANDS: bot: K, human: J
Folded round: 1
outcome all: tensor([0.3779, 0.0000], grad_fn=<CatBackward0>)
Loss: 51.086578369140625
===== Epoch 3 =====
HANDS: bot: Q, human: K
Folded round: 1
outcome all: tensor([0.3862, 0.0000], grad_fn=<CatBackward0>)
Loss: 1.0757286548614502
===== Epoch 4 =====
HANDS: bot: Q, human: K
Folded round: 1
outcome all: tensor([0.3867, 0.0000], grad_fn=<CatBackward0>)
Loss: 1.0750821828842163
===== Epoch 5 =====
HANDS: bot: K, human: Q
Folded round: 1
outcome all: tensor([0.3795, 0.0000], grad_fn=<CatBackward0>)
Loss: 51.08442306518555
===== Epoch 6 =====
HANDS: bot: Q, human: K
Folded round: 1
ou

## Loss functions
A loss function is takes the output of a neural network and quantifies it against
the expected outcome. This quantification gives inreturn a way of determining how wrong
the model(neural network) got. 
By minimizing loss from the function the model will be better at predicting what the 
execpted output should be. 
Loss functions are plurar because there are many different functions that tender to a specific tagreted model predictive output and exepcted output.

In the kuhn poker model the question the model has to learn is the card I posses a good card.
And thefore should the model fold or call?

A fold or call question is the output the model returns and this is refered to as a classification problem. But not just a classification due to the seens of the model only having the choose between fold or call is a binary classifictation.
Then a prefered loss function choosen here is called Binay Cross-Entropy.

### Binay Cross-Entropy
Binay Cross-Entropy or BCELoss for short in Pytorch library.
BCELoss is a special case of Cross-Entropy loss when there are only two classifactations.
Binay Cross-Entropy presupooses that x and y are probabilites and strictily between 0 and 1.
BCELoss creates a standard on which a judgement for the loss will be based on.
The standard is measured between the target probabilities and the input probabilities.
The mathematics of BCELoss is:
$$L = \{l_1, ..., l_N\}^T$$ 
* Where:
    - N is the batch size. 
    - $l_n$ is: 
$$ l_n = -w_n[y_n * log(x_n) + (1 - y_n) * log(1-x_n)]$$

Observe that if $x_n$ is either 0 or 1 by the fact of it being a binary classification.
If either $y_n=0$ or $(1 - y_n) = 0$ then on of the terms in $l_n$ will be zero.
Example is $y_n = 0$ then $l_n = -w[0 * log(x_n) + (1 - 0) * log(1-x_n)]$

The BCELoss is designed for binary classifactation when it exsist a distribution of measured
uncertainty to classify.

## Testing

In [182]:
print(10*"=", "Playing", "="*10)
#NeuralNetwork(n_inputs=2, h_layer=3, n_neurons=7, n_output=2)

kuhn = Kuhn()
matches = 50
w, l, d = 0, 0, 0

for _ in range(matches):
    kuhn.new_match("human", "bot")
    
    hands = kuhn.pre_showdown()
    
    for i in range(2):
        X = torch.tensor([Kuhn.deck_value[hands["bot"]], i], dtype=torch.float)
        output = net(X)
        
        # FOLD
        if clear_output(output) == [0, 1]:
            winner = "fold"
            d += 1
            break

        if i == 1: # -> winner
            winner = kuhn.showdown()
            if winner == "bot":
                w += 1
            else:
                l += 1

    print(f"human: {hands['human']}, bot: {hands['bot']} and bot result", winner)

print(10*"=", "Summary", "="*10)
print('win : ',w)
print('draw: ',d)
print('loss: ',l)

========== Playing ==========
human: K, bot: J and bot result human
human: K, bot: J and bot result human
human: J, bot: K and bot result bot
human: Q, bot: J and bot result human
human: J, bot: Q and bot result bot
human: J, bot: Q and bot result bot
human: J, bot: K and bot result bot
human: K, bot: J and bot result human
human: J, bot: Q and bot result bot
human: K, bot: J and bot result human
human: K, bot: J and bot result human
human: J, bot: Q and bot result bot
human: K, bot: J and bot result human
human: J, bot: Q and bot result bot
human: Q, bot: J and bot result human
human: K, bot: J and bot result human
human: Q, bot: K and bot result bot
human: Q, bot: K and bot result bot
human: K, bot: Q and bot result human
human: Q, bot: J and bot result human
human: K, bot: Q and bot result human
human: K, bot: J and bot result human
human: J, bot: K and bot result bot
human: K, bot: Q and bot result human
human: K, bot: Q and bot result human
human: J, bot: Q and bot result bot
huma